In [1]:
import os
import sys
import tensorflow as tf
import keras
import keras.preprocessing.image
import json
import random
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# keras_retinanet imports
from keras_retinanet import losses
from keras_retinanet import layers
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# sys.path.append('/root/amol/product_detection/keras-retinanet/keras_retinanet/preprocessing/')
# from csv_generator import CSVGenerator
# from ..models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.keras_version import check_keras_version

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

In [4]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

In [5]:
def create_generators(annotations,classes,batch_size=1,val_annotations=''):
    # create random transform generator for augmenting training data
    transform_generator = random_transform_generator(flip_x_chance=0.5)

    
    
    train_generator = CSVGenerator(
        annotations,
        classes,
        transform_generator=transform_generator,
        batch_size=batch_size
    )


    if val_annotations:
        validation_generator = CSVGenerator(
            val_annotations,
            classes,
            batch_size=batch_size
        )
    else:
        validation_generator = None
    

    return train_generator, validation_generator

In [6]:
def create_callbacks(model, training_model, prediction_model, validation_generator, args):
    callbacks = []

    # save the prediction model
    if args.snapshots:
        # ensure directory created first; otherwise h5py will error after epoch.
        os.makedirs(args.snapshot_path, exist_ok=True)
        checkpoint = keras.callbacks.ModelCheckpoint(
            os.path.join(
                args.snapshot_path,
                '{backbone}_{dataset_type}_{{epoch:02d}}.h5'.format(backbone=args.backbone, dataset_type=args.dataset_type)
            ),
            verbose=1
        )
        checkpoint = RedirectModel(checkpoint, prediction_model)
        callbacks.append(checkpoint)

    tensorboard_callback = None

    if args.tensorboard_dir:
        tensorboard_callback = keras.callbacks.TensorBoard(
            log_dir                = args.tensorboard_dir,
            histogram_freq         = 0,
            batch_size             = args.batch_size,
            write_graph            = True,
            write_grads            = False,
            write_images           = False,
            embeddings_freq        = 0,
            embeddings_layer_names = None,
            embeddings_metadata    = None
        )
        callbacks.append(tensorboard_callback)

    if args.evaluation and validation_generator:
        if args.dataset_type == 'coco':
            from ..callbacks.coco import CocoEval

            # use prediction model for evaluation
            evaluation = CocoEval(validation_generator)
        else:
            evaluation = Evaluate(validation_generator, tensorboard=tensorboard_callback)
        evaluation = RedirectModel(evaluation, prediction_model)
        callbacks.append(evaluation)

    callbacks.append(keras.callbacks.ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'auto',
        epsilon  = 0.0001,
        cooldown = 0,
        min_lr   = 0
    ))

    return callbacks

In [7]:
def model_with_weights(model, weights, skip_mismatch):
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model

In [8]:
def create_models(backbone_retinanet, backbone, num_classes, weights, multi_gpu=0, freeze_backbone=False):
    modifier = freeze_model if freeze_backbone else None

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        with tf.device('/cpu:0'):
            model = model_with_weights(backbone_retinanet(num_classes, backbone=backbone, nms=False, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)

        # append NMS for prediction only
        classification   = model.outputs[1]
        detections       = model.outputs[2]
        boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
        detections       = layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
        prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])
    else:
        model            = model_with_weights(backbone_retinanet(num_classes, backbone=backbone, nms=True, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model   = model
        prediction_model = model

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )

    return model, training_model, prediction_model

In [9]:
annotations = '/root/thomas/mask/data/annotations.csv'
classes = '/root/thomas/mask/data/classID.csv'
batch_size = 8
val_annotations = None
weights = download_imagenet('resnet50')


In [10]:
train_generator, validation_generator = create_generators(annotations, classes, batch_size, val_annotations)

In [11]:
model, training_model, prediction_model = create_models(backbone_retinanet=retinanet,
                                                        backbone='resnet50',
                                                        num_classes=train_generator.num_classes(),
                                                        weights=weights)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.


In [12]:
# # create the callbacks
# callbacks = create_callbacks(
#     model,
#     training_model,
#     prediction_model,
#     validation_generator,
#     args=args)


In [ ]:
# start training
training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=1000,
    epochs=20,
    verbose=1,
    # callbacks=callbacks,
)

Epoch 1/20
 398/1000 [==========>...................] - ETA: 14:55 - loss: 3.8257 - regression_loss: 3.1963 - classification_loss: 0.6294